# Data acquisition in arbok

### Arbok module imports

In [1]:
import numpy as np
from arbok.samples.aurora.configs.aurora_qua_config import aurora_qua_config
from arbok.core.program import Program
from arbok.core.sample import Sample

2023-09-18 10:25:48,679 - qm - INFO     - Starting session: f463f5a2-f0ce-46a2-9200-e491e9555cdb


Instanciating sample and measurement objects

In [2]:
aurora = Sample('aurora', aurora_qua_config)
ESR_search = Program('ESR_search', sample = aurora)

In [3]:
from arbok.samples.aurora.configs.aurora_qua_config import aurora_qua_config

from arbok.samples.aurora.initialization.odd_6_dot_init import InitOdd6dot
from arbok.samples.aurora.control.rabi import Rabi
from arbok.samples.aurora.readout.psb_6_dot_read import PSB6dotRead

Creating Subsequences and adding them to the measurement object `ESR_search`

## Rabi chevron sequence creation

In [4]:
init6 = InitOdd6dot('init6')
read6 = PSB6dotRead('read6')
rabi6 = Rabi('rabi6')

In [5]:
RabiChevron = Program('RabiChevron', sample = aurora)

RabiChevron.add_subsequence(init6)
RabiChevron.add_subsequence(rabi6)
RabiChevron.add_subsequence(read6)

In [6]:
f_detune=np.arange(32.2e6,34.2e6,0.05e6, dtype = int)
print(len(f_detune))
t_list = np.arange(4,30e3/4,200, dtype = int)
print(len(t_list))

40
38


In [7]:
RabiChevron.settables = [
    RabiChevron.rabi6.IfQ1,
    RabiChevron.rabi6.tRabi]
RabiChevron.setpoints_grid = [
    f_detune,
    t_list]
RabiChevron.gettables = [
    RabiChevron.read6.diff_set1_state,
    RabiChevron.read6.diff_set2_state]

RabiChevron.connect_OPX(host_ip= '192.168.0.3')

2023-09-18 10:25:49,358 - qm - INFO     - Performing health check
2023-09-18 10:25:49,393 - qm - INFO     - Health check passed


In [8]:
rabi_chevron_program = RabiChevron.get_program()

Adding qua <class 'int'> variable for IfQ1
Adding qua <class 'int'> variable for tRabi
Adding qua sweep loop for tRabi
Adding qua sweep loop for IfQ1


In [9]:
RabiChevron.settables

[<arbok.core.sequence_parameter.SequenceParameter: IfQ1 at 1432949671504>,
 <arbok.core.sequence_parameter.SequenceParameter: tRabi at 1432945123536>]

In [10]:
RabiChevron.prepare_gettables()

<Arrays, shape: (40,)>
<Arrays, shape: (40,)>
<Arrays, shape: (38,)>
<Arrays, shape: (38,)>


In [11]:
RabiChevron.rabi6.IfQ1.vals

<Arrays, shape: (40,)>

In [12]:
RabiChevron.read6.diff_set2_state.setpoints[0].vals

<Arrays, shape: (40,)>

In [13]:
RabiChevron.read6.diff_set2_state.vals

<Arrays, shape: (40, 38)>

In [14]:
from qcodes.validators import Arrays
setpoints_shape_list = []

for sp in RabiChevron.read6.diff_set2_state.setpoints:
    if not isinstance(sp.vals, Arrays):
        raise ValueError(
            f"Can only validate shapes for parameters "
            f"with Arrays validator. {sp.name} is "
            f"a setpoint vector but does not have an "
            f"Arrays validator"
        )
    if sp.vals.shape_unevaluated is not None:
        setpoints_shape_list.extend(sp.vals.shape_unevaluated)
    else:
        setpoints_shape_list.append(sp.vals.shape_unevaluated)

In [15]:
setpoints_shape_list

[40, 38]

In [16]:
#RabiChevron.get_running_qm_job()
RabiChevron.run(rabi_chevron_program)

2023-09-18 10:25:52,680 - qm - INFO     - Sending program to QOP for compilation
2023-09-18 10:26:05,945 - qm - INFO     - Executing program


In [17]:
RabiChevron.opx.get_running_job()

## How to measure: Rabi Chevron

In [18]:
RabiChevron.read6.diff_set2_state.setpoints[0].vals

<Arrays, shape: (40,)>

In [19]:
setpoints_shape_list

[40, 38]

##  Rabi Chevron in QCoDeS framework

In [20]:
import os
import qcodes as qc
from qcodes.dataset import (
    LinSweep,
    Measurement,
    dond,
    experiments,
    initialise_or_create_database_at,
    load_by_run_spec,
    load_or_create_experiment,
    plot_dataset,
)

## Using interactive widget
from qcodes.interactive_widget import experiments_widget

## Dummy instruments for generating synthetic data
from qcodes.tests.instrument_mocks import (
    DummyInstrument,
    DummyInstrumentWithMeasurement,
)

In [21]:
tutorial_db_path = os.path.join(os.getcwd(), "example_dbs/experiments_for_15_mins.db")
initialise_or_create_database_at(tutorial_db_path)
load_or_create_experiment(experiment_name='Rabi_Chevron', sample_name="aurora")

Rabi_Chevron#aurora#2@C:\Users\labuser\GitRepos\arbok\arbok\examples\example_dbs\experiments_for_15_mins.db
-----------------------------------------------------------------------------------------------------------
38-results-1-rabi6_IfQ1,rabi6_tRabi,diff_set1_state,diff_set2_state-0
39-results-2-rabi6_IfQ1,rabi6_tRabi,diff_set1_state,diff_set2_state,iteration-0
40-results-3-rabi6_IfQ1,rabi6_tRabi,diff_set1_state,diff_set2_state,RabiChevron_iteration-0

In [22]:
qc.logger.start_all_logging()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\labuser\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\labuser\.qcodes\logs\230918-12748-qcodes.log


In [23]:
qc.config.core.db_location

'C:\\Users\\labuser\\GitRepos\\arbok\\arbok\\examples\\example_dbs/experiments_for_15_mins.db'

In [24]:
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

jobs = bg.BackgroundJobManager()
jobs.new(inspectr.main, qc.config.core.db_location)

<BackgroundJob #0: <function main at 0x0000014DC4387740>>

In [25]:
meas = Measurement()
# meas.register_parameter(a.spectrum)

# with meas.run() as datasaver:
#     datasaver.add_result((a.freq_axis, a.freq_axis()),
#                          (a.spectrum, a.spectrum()))
#     dataset = datasaver.dataset

In [27]:
out = RabiChevron.read6.diff_set2_state.get()

FETC 3040 3040 diff_set2_state
READ 3040 3040
2023-09-18 10:26:48,898 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1520,)


In [29]:
out.shape

(40, 38)

In [ ]:
type()

In [ ]:
#RabiChevron.run_qc_measurement(meas, 100)

In [ ]:
x = tuple(len(x) for x in RabiChevron.setpoints_grid)

In [ ]:
x